In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from google.colab import drive, files
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm


In [ ]:
uploaded=files.upload()

In [ ]:
for filename in uploaded.keys():
  df=pd.read_excel(filename)

grouped =df.groupby('track_id')


In [ ]:
# Filter
selected_ids=grouped.filter(lambda x: len(x)>=280)['track_id'].unique()
filtered_data=df[df['track_id'].isin(selected_ids)]

# Extract rows where t == 288
result = filtered_data[filtered_data['t'] == 288]

# Filter to include rows only when fil1's track_id is in result's track_id
fil1 = filtered_data[filtered_data['track_id'].isin(result['track_id'].unique())]

# Extract rows where t != 288
input = fil1[fil1['t'] != 288]

In [ ]:
col = input.columns.get_loc('total_int_nuc')

In [ ]:
col = 'total_int_nuc'

# Function to process each track_id
def process_track_id(track_id, input, col):
    input0 = input[input['track_id'] == track_id]
    sum_MI = input0[col].sum()
    mean_MI = input0[col].mean()
    sd_MI = input0[col].std()
    min_t = input0.loc[input0[col].idxmin(), 't']
    max_t = input0.loc[input0[col].idxmax(), 't']

    start_value = input0.loc[input0['t'].idxmin(), col]
    end_value = input0.loc[input0['t'].idxmax(), col]

    min_value = input0[col].min()
    initial_min_fold = start_value / min_value
    final_min_fold = end_value / min_value

    fc_nomean = end_value / start_value
    mean_start = input0[col].iloc[:5].mean()
    fc_mean = end_value / mean_start

    mean_deri = np.mean(np.abs(np.diff(input0[col])))

    # Perform linear regression
    X = input0[['t']]
    y = input0[col]
    lm = LinearRegression().fit(X, y)
    slope = lm.coef_[0]
    y_inter = lm.intercept_

    # Calculate integral of the linear model
    integral_lm = ((slope * (input0['t'].min() + input0['t'].max()) + 2 * y_inter) *
                   (input0['t'].max() - input0['t'].min()) / 2)

    # Compile results into a list
    input_new = [track_id, sum_MI, mean_MI, sd_MI, mean_deri, min_t, max_t,
                 start_value, end_value, fc_nomean, initial_min_fold, final_min_fold,
                 mean_start, fc_mean, slope, integral_lm]

    return input_new

# Process each unique track_id and compile the results
input_new = [process_track_id(track_id, input, col) for track_id in input['track_id'].unique()]

# Convert the results to a DataFrame
input_new_df = pd.DataFrame(input_new, columns=[
    'track_id', 'sum_MI', 'mean_MI', 'sd_MI', 'mean_deri', 'min_t', 'max_t',
    'start_value', 'end_value', 'foldchange_nomean', 'initial_min_fold', 'final_min_fold',
    'mean_start', 'foldchange_mean', 'slope', 'integral_lm'
])

# print(input_new_df)


In [ ]:
input_new_df.set_index('track_id', inplace=True)
result.set_index('track_id', inplace=True)

In [ ]:
dat = pd.concat([input_new_df, result['total_IL8']], axis=1)

# Rename the last column to 'result'
dat.columns.values[-1] = 'result'
# print(dat)

In [ ]:
## don't use all of the input, select the most relavent and non-conflicting ones

y = dat['result']

# Define the independent variables (predictors)
X = dat[['mean_MI', 'sd_MI', 'mean_deri', 'start_value', 'foldchange_nomean',
         'slope', 'integral_lm', 'min_t', 'max_t', 'final_min_fold']]

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the linear model
lm_total = sm.OLS(y, X).fit()

# Output the summary of the linear model
print(lm_total.summary())

                            OLS Regression Results                            
Dep. Variable:                 result   R-squared:                       0.546
Model:                            OLS   Adj. R-squared:                  0.520
Method:                 Least Squares   F-statistic:                     21.50
Date:                Thu, 01 Aug 2024   Prob (F-statistic):           5.97e-26
Time:                        21:56:45   Log-Likelihood:                -2669.3
No. Observations:                 190   AIC:                             5361.
Df Residuals:                     179   BIC:                             5396.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              3.495e+05   1.56e+0